In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [17]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [18]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [19]:
import wandb

In [20]:
from tqdm import tqdm

In [21]:
EPOCHS = 100

In [22]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [23]:
preds

tensor([3.7035e-02, 1.4371e-02, 1.6785e-02, 9.8762e-01, 4.1225e-02, 1.4806e-03,
        9.8883e-01, 3.8394e-03, 5.6228e-05, 2.3920e-02, 9.7746e-01, 9.9119e-01,
        3.4683e-03, 9.6440e-01, 5.4233e-03, 6.9786e-01, 6.8937e-03, 9.0952e-03,
        9.9508e-01, 2.7360e-03, 3.3192e-02, 3.0701e-02, 2.7928e-03, 9.7720e-01,
        7.4061e-03, 9.8003e-01, 1.6891e-02, 9.8265e-01, 4.9796e-02, 3.0093e-02,
        1.4683e-03, 9.8141e-01, 1.8749e-03, 9.3589e-03, 3.0833e-03, 7.3436e-03,
        2.5329e-02, 9.8970e-01, 2.0301e-04, 1.2641e-04, 1.6729e-01, 1.7698e-02,
        2.1872e-04, 1.2164e-02, 4.1592e-02, 1.1803e-03, 3.3742e-03, 1.8250e-01,
        8.5229e-03, 9.8196e-01, 8.3979e-04, 8.5159e-01, 9.7723e-04, 9.2131e-01,
        2.6449e-04, 9.7212e-01, 1.2548e-03, 6.2677e-01, 1.5177e-02, 1.0060e-02,
        8.3739e-01, 1.1186e-03, 4.1676e-03, 8.8335e-03, 3.9794e-01, 9.6608e-01,
        3.1726e-01, 9.8333e-01, 1.4938e-04, 1.1333e-02, 1.4410e-02, 7.3301e-04,
        7.4415e-03, 9.9313e-01, 5.3082e-

In [24]:
torch.round(preds)

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 

In [25]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [26]:
torch.round(preds)[:12]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [27]:
EPOCHS = 125

In [28]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [29]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [30]:
torch.round(preds)[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [31]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [32]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [33]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [34]:
preds[0]

tensor(0.9987, device='cuda:0', grad_fn=<SelectBackward>)

In [35]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [36]:
EPOCHS = 112

In [37]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [38]:
preds

tensor([0.9880, 0.9966, 0.9928, 0.9993, 0.9945, 0.9985, 0.9992, 0.9986, 0.9604,
        0.9843, 0.9983, 0.9989, 0.9810, 0.9966, 0.9971, 0.9914, 0.9979, 0.9936,
        0.9994, 0.9958, 0.9898, 0.9890, 0.9988, 0.9989, 0.9984, 0.9992, 0.9932,
        0.9990, 0.9984, 0.9966, 0.9428, 0.9990, 0.9857, 0.9972, 0.0142, 0.9867,
        0.9843, 0.9986, 0.0058, 0.9897, 0.9982, 0.9985, 0.6632, 0.9665, 0.9915,
        0.9952, 0.9930, 0.9137, 0.9967, 0.9988, 0.0649, 0.9993, 0.9887, 0.9982,
        0.0165, 0.9994, 0.9919, 0.9751, 0.9603, 0.9991, 0.9989, 0.6089, 0.4025,
        0.9986, 0.9984, 0.9970, 0.9992, 0.9993, 0.9719, 0.4760, 0.9992, 0.4174,
        0.9928, 0.9998, 0.9792, 0.9997, 0.9997, 0.0263, 0.9977, 0.9995, 0.9938,
        0.9990, 0.9096, 0.9843, 0.9994, 0.9937, 0.9603, 0.0387, 0.9990, 0.9992,
        0.9969, 0.9883, 0.9962, 0.9999, 0.9930, 0.0889, 0.9987, 0.9982, 0.9687,
        0.9825, 0.9953, 0.9863, 0.9995, 0.9993, 0.9990, 0.9988, 0.9301, 0.8482,
        0.9995, 0.9993, 0.0379, 0.9988, 

In [39]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [40]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [41]:
EPOCHS = 75

In [42]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()